In [ ]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.ticker import PercentFormatter

import torch
import torch.nn.functional as F
import cv2

import os
from tqdm import tqdm

from layers import disp_to_depth
from utils import readlines
import datasets

In [ ]:
LOG_DIR = './history'
VERSION = 'model_24_09_lite'

In [ ]:
def show_image(dataset, index, show_gt = False, depth_pred = None):
    item = dataset.__getitem__(index)
    color_im = item[('color', 0, 0)].permute(1,2,0).cpu().detach().numpy() if type(item[('color', 0, 0)]) is torch.Tensor else item[('color', 0, 0)]
    depth_map = item["depth_gt"].permute(1,2,0).cpu().detach().numpy() if type(item["depth_gt"]) is torch.Tensor else item["depth_gt"]
    
    plt.figure(figsize=(18,6))
    plt.subplot(1,3,1)
    plt.title("Input")
    plt.imshow(color_im)
    
    if show_gt:
        plt.subplot(1,3,2)
        plt.title("Ground truth")
        plt.imshow(depth_map)

    if depth_pred is not None:
        plt.subplot(1,3,3)
        plt.title("Prediction")
        plt.imshow(depth_pred.permute(1,2,0).cpu().detach().numpy() if type(depth_pred) is torch.Tensor else depth_pred)
    
    plt.show()  
    
def show_compare_result(img_color, depth_pred_1, depth_pred_2):
    plt.figure(figsize=(18,6))
    plt.subplot(1,3,1)
    plt.title("Input")
    plt.imshow(img_color)
    
    plt.subplot(1,3,2)
    plt.title("Depth_pred_1")
    plt.imshow(depth_pred_1)

    plt.subplot(1,3,3)
    plt.title("Depth_pred_2")
    plt.imshow(depth_pred_2)
    
    plt.show() 
    
def change_color(new_pred_disp):
    vmax = np.percentile(new_pred_disp, 95)
    normalizer = mpl.colors.Normalize(vmin=new_pred_disp.min(), vmax=vmax)
    mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
    colormapped_im = (mapper.to_rgba(new_pred_disp)[:, :, :3] * 255).astype(np.uint8)
    return colormapped_im

In [ ]:
def compute_errors(gt, pred):
    """Computation of error metrics between predicted and ground truth depths
    """
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return [abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3]

In [ ]:
def evaluate_per_pixel(gt_depth, pred_disp, opt):
    gt_height, gt_width = gt_depth.shape[:2]
    pred_disp = cv2.resize(pred_disp, (gt_width, gt_height))
    pred_depth = 1 / pred_disp


    if opt.eval_split == "eigen":
        mask = np.logical_and(gt_depth > MIN_DEPTH, gt_depth < MAX_DEPTH)

        crop = np.array([0.40810811 * gt_height, 0.99189189 * gt_height,
                         0.03594771 * gt_width,  0.96405229 * gt_width]).astype(np.int32)
        crop_mask = np.zeros(mask.shape)
        crop_mask[crop[0]:crop[1], crop[2]:crop[3]] = 1
        mask = np.logical_and(mask, crop_mask)
    else:
        mask = gt_depth > 0

    pred_depth = pred_depth[mask]
    gt_depth = gt_depth[mask]

    pred_depth *= opt.pred_depth_scale_factor
    if not opt.disable_median_scaling:
        ratio = np.median(gt_depth) / np.median(pred_depth)
#         ratios.append(ratio)
        pred_depth *= ratio

    pred_depth[pred_depth < MIN_DEPTH] = MIN_DEPTH
    pred_depth[pred_depth > MAX_DEPTH] = MAX_DEPTH
    
    abs_rel = np.abs(pred_depth - gt_depth) / gt_depth
    
    return abs_rel, gt_depth

# Read opt.json

In [ ]:
import json
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

LOG_DIR = './history'
VERSION = 'semi_sup_28_09'        

opt = json.load(open(f"{LOG_DIR}/{VERSION}/models/opt.json"))
opt = Struct(**opt)
print(opt.__dict__.items())

# Read image

In [ ]:
splits_dir = "./splits"
img_ext = '.png'
test_filenames = readlines(os.path.join(splits_dir, opt.eval_split, "test_files.txt"))

test_dataset = datasets.KITTIRAWDataset(opt.data_path, test_filenames,
                                           192, 640,
                                           [0], 4, is_train=False, img_ext=img_ext)

fpath = os.path.join("./", "splits", opt.split, "{}_files.txt")
train_filenames = readlines(fpath.format("train"))
val_filenames = readlines(fpath.format("val"))

val_dataset = datasets.KITTIRAWDataset(opt.data_path, val_filenames, 192, 640, [0], 4, is_train=False, img_ext=img_ext)
train_dataset = datasets.KITTIRAWDataset(opt.data_path, train_filenames, 192, 640, [0], 4, is_train=False, img_ext=img_ext)

In [ ]:
max_depths = []
for i in range(len(train_filenames)//100):
    max_depths.append(train_dataset.__getitem__(i)['depth_gt'].max())
qlst = np.percentile(max_depths, [0,25,50,75,100])
print(qlst)

In [ ]:
for i in range(100):
    show_image(val_dataset, i)

# Read training_process.txt

In [ ]:
def create_dict(score_line):
    score_dict = {}
    for idx, line in enumerate(score_line):
        string_scores = line.split("|")
        for string_score in string_scores:
            if "\n" in string_score:
                continue
            
            name, value = string_score.strip().split() if "Batch" in string_score else string_score.split(":")[-2:]
            if "Loss sup_loss/0" in name:
                name = "sup_loss/0"
            if(idx == 0):
                score_dict[name.strip()] = [value.strip() if "Epoch" in value else float(value.strip())]
            else:
                score_dict[name.strip()].append(value.strip() if "Epoch" in value else float(value.strip()))
    return score_dict

def compare_score_plot(dic1, dic2, field):
    print(field)
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.plot(dic1[field])
    plt.title("Train")
    
    plt.subplot(1,2,2)
    plt.plot(dic2[field])
    plt.title("Valid")
    
    plt.show()

In [ ]:
LOG_DIR = './history'
VERSION = 'the_last'

training_process_txt = []
with open(f"{LOG_DIR}/{VERSION}/training_process.txt") as f:
    training_process_txt = f.readlines()
    
valid_score_line = []
train_score_line = []
for line in training_process_txt:
    if "Validation" in line:
        valid_score_line.append(line)
    else:
        train_score_line.append(line)

train_score_dict = create_dict(train_score_line)
valid_score_dict = create_dict(valid_score_line)

In [ ]:
valid_score_pd = pd.DataFrame(valid_score_dict)
train_score_pd = pd.DataFrame(train_score_dict)
train_score_pd.columns

In [ ]:
compare_score_plot(train_score_pd, valid_score_pd, "sup_loss/0")
compare_score_plot(train_score_pd, valid_score_pd, "selsup_loss/0")
compare_score_plot(train_score_pd, valid_score_pd, "loss")
compare_score_plot(train_score_pd, valid_score_pd, "de/abs_rel")

# EA on test set

In [ ]:
"""Load predictions
"""
gt_path = os.path.join("./splits", opt.eval_split, "gt_depths.npz")
gt_depths = np.load(gt_path, allow_pickle=True, fix_imports=True, encoding='latin1')["data"]

old_model_path = "./history/mobile_hr/models/weights_14"
new_model_path = "./history/model_26_09_lite/models_1/weights_1"
old_pred_disps = np.load(f'{old_model_path}/disps_eigen_split.npy', allow_pickle=True, fix_imports=True, encoding='latin1')
new_pred_disps = np.load(f'{new_model_path}/disps_eigen_split.npy', allow_pickle=True, fix_imports=True, encoding='latin1')
max_depth_pred  = np.load(f'{new_model_path}/max_depth_eigen_split.npy', allow_pickle=True, fix_imports=True, encoding='latin1')

In [ ]:
max_depth_gt = np.array([x.max() for x in gt_depths])
percentile_values = np.percentile(max_depth_gt, [0,5,50,75,100])
print(percentile_values)
percentile_values = np.percentile(max_depth_pred, [0,5,50,75,100])
print(percentile_values)

In [ ]:
"""Compute metric 
"""
MIN_DEPTH = 1e-3
MAX_DEPTH = 80

def compute_metric(gt_depths, pred_disps, opt):
    errors = []
    abs_rels = []
    ratios = []
    rs_gt_depths = []

    for i in tqdm(range(pred_disps.shape[0])):

        gt_depth = gt_depths[i]
        gt_height, gt_width = gt_depth.shape[:2]

        pred_disp = pred_disps[i]
        pred_disp = cv2.resize(pred_disp, (gt_width, gt_height))
        pred_depth = 1 / pred_disp
        
                                
        if opt.eval_split == "eigen":
            mask = np.logical_and(gt_depth > MIN_DEPTH, gt_depth < MAX_DEPTH)

            crop = np.array([0.40810811 * gt_height, 0.99189189 * gt_height,
                             0.03594771 * gt_width,  0.96405229 * gt_width]).astype(np.int32)
            crop_mask = np.zeros(mask.shape)
            crop_mask[crop[0]:crop[1], crop[2]:crop[3]] = 1
            mask = np.logical_and(mask, crop_mask)
        else:
            mask = gt_depth > 0

        pred_depth = pred_depth[mask]
        gt_depth = gt_depth[mask]

        pred_depth *= opt.pred_depth_scale_factor
        if not opt.disable_median_scaling:
            ratio = np.median(gt_depth) / np.median(pred_depth)
            ratios.append(ratio)
            pred_depth *= ratio

        pred_depth[pred_depth < MIN_DEPTH] = MIN_DEPTH
        pred_depth[pred_depth > MAX_DEPTH] = MAX_DEPTH

#         if opt.eval_return_result:
#             pred_depths.append(copy.deepcopy(pred_depth))
#             rs_gt_depths.append(copy.deepcopy(gt_depth))

        error = compute_errors(gt_depth, pred_depth)
        abs_rels.append(error[0])
        errors.append(error)

    if not opt.disable_median_scaling:
        ratios = np.array(ratios)
        med = np.median(ratios)
        print(" Scaling ratios | med: {:0.3f} | std: {:0.3f}".format(med, np.std(ratios / med)))
    
    mean_errors = np.array(errors).mean(0)
#     if opt.predict_visibility:
#         max_depth_mse = MSE(gt_max_depths, pred_max_depths)
#     else:
#         max_depth_mse = -1
    mean_errors = mean_errors.tolist()
#     mean_errors.append(max_depth_mse)
    print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3"))
    print(("&{: 8.3f}  " * 7).format(*mean_errors) + "\\\\")
    print("\n-> Done!")
    
    return errors, abs_rels, ratios

In [ ]:
old_errors, old_abs_rels, old_ratios = compute_metric(gt_depths, old_pred_disps, opt)
new_errors, new_abs_rels, new_ratios = compute_metric(gt_depths, new_pred_disps, opt)

## Compare depth_map

In [ ]:
"""Visualization
"""

rank_idx = np.argsort(np.array(old_abs_rels))
for i in range(len(rank_idx)):
    idx = rank_idx[i]
    
    if (old_abs_rels[idx] - new_abs_rels[idx]) < 0.03 or gt_depths[idx].max() > 79:
        continue
    
    img_color = test_dataset.__getitem__(idx)[('color_aug', 0, 0)].unsqueeze(0)
    img_color = F.interpolate(img_color, [375,1242], mode = 'bilinear', align_corners = False).squeeze(0)
    img_color = img_color.permute(1,2,0).numpy()
    
#     if gt_depths[idx].max() > 65 or (old_abs_rels[idx] < new_abs_rels[idx]):
#         continue
    
    print(f"Max_depth: Ground truth {gt_depths[idx].max()} - Prediction {max_depth_pred[idx]}")
    plt.figure(figsize=(18,6))
    plt.subplot(1,3,1)
    plt.title(f"Input")
    plt.imshow(img_color)
    plt.axis("off")
    
    plt.subplot(1,3,2)
    plt.title("Mobile-HR-Depth | Abs_rel: {:.4f}".format(old_abs_rels[idx]))
    plt.imshow(change_color(old_pred_disps[idx]))
    plt.axis("off")
    
    plt.subplot(1,3,3)
    plt.title("VisiDepth | Abs_rel: {:.4f}".format(new_abs_rels[idx]))
    plt.imshow(change_color(new_pred_disps[idx]))
    plt.axis("off")
    plt.show() 

## Plot metric + bins of depth

In [ ]:
idxs = []
rank_idx = np.argsort(np.array(old_abs_rels))
for i in range(len(rank_idx)):
    idx = rank_idx[i]
    
    if (old_abs_rels[idx] - new_abs_rels[idx]) > 0.00:
        idxs.append(idx)
len(idxs)

In [ ]:
num_bins = 16
len_bin = 80 // num_bins

new_abs_rel_bins_all = [[] for i in range(num_bins)]
old_abs_rel_bins_all = [[] for i in range(num_bins)]
gt_depth_1d_all = []

for idx in idxs:
    gt_depth = gt_depths[idx]
    new_pred_disp = new_pred_disps[idx]
    old_pred_disp = old_pred_disps[idx]

    new_abs_rel, gt_depth_1d = evaluate_per_pixel(gt_depth, new_pred_disp, opt)
    old_abs_rel, gt_depth_1d = evaluate_per_pixel(gt_depth, old_pred_disp, opt)

    values = np.arange(0,80+len_bin, len_bin)
#     new_abs_rel_bins = np.zeros((num_bins,))
#     old_abs_rel_bins = np.zeros((num_bins,))

    for i in range(len(values)-1):
        mask = np.logical_and(gt_depth_1d > values[i], gt_depth_1d < values[i+1])
        if mask.sum() > 0:
            new_abs_rel_bins_all[i].append(np.mean(new_abs_rel[mask]))
            old_abs_rel_bins_all[i].append(np.mean(old_abs_rel[mask]))
    gt_depth_1d_all.append(gt_depth_1d)

gt_depth_1d_all = np.concatenate(gt_depth_1d_all)
new_abs_rel_bins = [np.mean(x) if len(x) > 0 else 0 for x in new_abs_rel_bins_all]
old_abs_rel_bins = [np.mean(x) if len(x) > 0 else 0 for x in old_abs_rel_bins_all]

In [ ]:
"""Visualize histogram + line
"""
old_name = "Mobile-HR-Depth (abs_rel)"
new_name = "VisiDepth (abs_rel)"

fig, ax1 = plt.subplots(figsize=(8,4))
ax1.set_xlabel("depth (m)")

ax2 = ax1.twinx()

# for histogram
ax1.hist(gt_depth_1d_all, bins = num_bins, range=[0,80], weights = np.ones(len(gt_depth_1d_all)) / len(gt_depth_1d_all), edgecolor='black', color="lightgrey")
# for rect in ax.patches:
#     height = rect.get_height()
#     ax1.annotate('{:.1f}%'.format(100*height), xy=(rect.get_x()+rect.get_width()/2, height), 
#                 xytext=(0, 5), textcoords='offset points', ha='center', va='bottom', fontsize=8)
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position("right")
ax1.yaxis.set_major_formatter(PercentFormatter(1))
ax1.set_ylabel("Percentage (%)")

# for line
ax2.plot(values[:-1] + len_bin / 2, new_abs_rel_bins, label=new_name, marker='o')
ax2.plot(values[:-1] + len_bin / 2, old_abs_rel_bins, label=old_name, marker='o')
ax2.legend(loc = 'center right')
ax2.yaxis.tick_left()
ax2.yaxis.set_label_position("left")
ax2.set_ylabel("Abs_rel")

#plt.show()
plt.savefig("./history/images/plot_histogram_0000.png")